In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import duckdb as dd
import polars as pl
import os
import glob
import shutil
import zipfile
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
import plotly.express as px
import librosa
from IPython.display import Audio
import pickle
from joblib import dump, load
from pathlib import Path
from datetime import datetime

In [2]:
audio_duration = 5

# Function to extract features from audio file
def extract_features(file_path, offset=0.0, duration=5.0):
    # Load audio file
    audio, sample_rate = librosa.load(path=file_path, offset=offset, duration=duration)
    """total_duration = librosa.get_duration(y=audio, sr=sample_rate)
    num_segments = int(total_duration // audio_duration)"""
    # Extract features using Mel-Frequency Cepstral Coefficients (MFCC)
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    # Flatten the features into a 1D array
    flattened_features = np.mean(mfccs.T, axis=0)
    return flattened_features

In [3]:
def audio_classification(file_path, offset=0.0, duration=5.0):
    extracted_features = extract_features(file_path, offset=offset, duration=duration).reshape(1, -1)
    y_predict = random_forest_model.predict_proba(extracted_features)
    
    return y_predict

In [4]:
random_forest_model = load('/kaggle/input/trained-audio-model-v-01/audio_classifier_model.joblib')

In [5]:
meta_data = pl.read_csv('../input/birdclef-2024/train_metadata.csv', low_memory=True)

In [6]:
bird_cols = list(meta_data['primary_label'].unique().sort())

In [54]:
# test_soundscapes = '/kaggle/input/birdclef-2024/unlabeled_soundscapes'
test_soundscapes = '/kaggle/input/birdclef-2024/test_soundscapes'

filenames_with_path = glob.glob(f"{test_soundscapes}/*.ogg")
# filenames = [('soundscape_' + os.path.basename(filename)) for filename in filenames_with_path]
filenames = [os.path.basename(filename) for filename in filenames_with_path]

print(len(filenames))

8444


In [ ]:
"""audio, sample_rate = librosa.load(path=test_soundscapes, offset=0.0, duration=5.0)
audio_full, sample_rate_full = librosa.load(path=test_soundscapes)"""

In [ ]:
# Audio(audio, rate=sample_rate)

In [ ]:
"""duration = len(audio) / sample_rate
time = np.arange(0, duration, 1/sample_rate)
plt.figure(figsize=(30, 4))
plt.plot(time, audio, color='blue')
plt.title('Audio Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plot = plt.show()"""

In [ ]:
"""n_fft = 500  # Number of FFT points 2048
hop_length = 50  # Hop length for STFT 512
stft = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
# Convert the magnitude spectrogram to decibels (log scale)
spectrogram = librosa.amplitude_to_db(np.abs(stft))
# Plot the spectrogram
plt.figure(figsize=(30, 6))
librosa.display.specshow(spectrogram, sr=sample_rate, hop_length=hop_length, x_axis='time', y_axis='linear')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.tight_layout()
plot = plt.show()"""

In [7]:
def audio_waveframe(file_path, offset=0.0, duration=5.0):
    # Load the audio file
    audio_data, sampling_rate = librosa.load(file_path, offset=offset, duration=duration)
    # Calculate the duration of the audio file
    duration = len(audio_data) / sampling_rate
    # Create a time array for plotting
    time = np.arange(0, duration, 1/sampling_rate)
    # Plot the waveform
    plt.figure(figsize=(30, 4))
    plt.plot(time, audio_data, color='blue')
    plt.title('Audio Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plot = plt.show()
    return plot

def spectrogram(file_path, offset=0.0, duration=5.0):
    # Compute the short-time Fourier transform (STFT)
    n_fft = 500  # Number of FFT points 2048
    hop_length = 50  # Hop length for STFT 512
    audio_data, sampling_rate = librosa.load(file_path, offset=offset, duration=duration)
    stft = librosa.stft(audio_data, n_fft=n_fft, hop_length=hop_length)
    # Convert the magnitude spectrogram to decibels (log scale)
    spectrogram = librosa.amplitude_to_db(np.abs(stft))
    # Plot the spectrogram
    plt.figure(figsize=(30, 6))
    librosa.display.specshow(spectrogram, sr=sampling_rate, hop_length=hop_length, x_axis='time', y_axis='linear')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.tight_layout()
    plot = plt.show()
    return plot

def audio_analysis(file_path, offset=0.0, duration=5.0):
    aw = audio_waveframe(file_path, offset, duration)
    spg = spectrogram(file_path, offset, duration)
    return aw, spg

In [ ]:
"""test_soundscapes = '/kaggle/input/birdclef-2024/unlabeled_soundscapes/100350238.ogg'
for j in range(0,241,5):
    predicted = audio_classification(test_soundscapes, offset=float(j), duration=5.0)
    filename_noext = test_soundscapes.replace('.ogg','')
    df['row_id'] = f'{filename_noext}_{j}'
    df[bird_cols] = predicted
    submission_df = pd.concat([submission_df,df]).reset_index(drop=True)"""

In [8]:
sample_submit = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
submit = pd.DataFrame(columns=sample_submit.columns)

submit

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1


In [61]:
# Test = True

# submission_df = pd.DataFrame(columns=['row_id']+bird_cols)
submission_df = pd.DataFrame(columns=sample_submit.columns)
print(submission_df.shape)
start = datetime.now()

for i in range(len(filenames_with_path)):
    now = datetime.now()
    for j in range(5,241,5):
        offset = j-5
        predicted = audio_classification(filenames_with_path[i], offset=float(offset), duration=5.0)
        filename_noext = filenames[i].replace('.ogg','')
        # print(f'{filename_noext}_{j}')
        submit['row_id'] = filename_noext+'_'+str(j)
        submit[bird_cols] = predicted.round(5)
        # print(submit['row_id'])
        submission_df = pd.concat([submission_df,submit]).reset_index(drop=True)
        later = datetime.now()
        difference = (later - now).total_seconds()
        
    print("processing of file {0} finished in {1} seconds".format(i, difference))
    
    """if i == 9:
        end = datetime.now()
        difference = (end - start).total_seconds()
        print("processing of 10 files finished in {0} seconds".format(difference))
        break"""

(0, 183)


/tmp/ipykernel_33/2613609158.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  submission_df = pd.concat([submission_df,submit]).reset_index(drop=True)


processing of file 0 finished in 3.569171 seconds
processing of file 1 finished in 3.834914 seconds
processing of file 2 finished in 3.565581 seconds
processing of file 3 finished in 3.720506 seconds
processing of file 4 finished in 3.849375 seconds
processing of file 5 finished in 3.60151 seconds
processing of file 6 finished in 3.810623 seconds
processing of file 7 finished in 3.759506 seconds
processing of file 8 finished in 3.675869 seconds
processing of file 9 finished in 3.689977 seconds
processing of 10 files finished in 37.082971 seconds


In [ ]:
# submission_df.sum(axis=1, numeric_only=True)

In [62]:
# submission_df.head(10)

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,1872382287_5,0.00,0.03,0.02,0.00,0.0,0.0,0.00,0.00,0.00,...,0.02,0.00,0.01,0.0,0.00,0.03,0.0,0.0,0.00,0.02
1,1872382287_10,0.00,0.01,0.01,0.00,0.0,0.0,0.01,0.00,0.00,...,0.01,0.00,0.01,0.0,0.00,0.04,0.0,0.0,0.01,0.02
2,1872382287_15,0.01,0.02,0.00,0.00,0.0,0.0,0.00,0.00,0.00,...,0.00,0.00,0.00,0.0,0.00,0.06,0.0,0.0,0.00,0.01
3,1872382287_20,0.00,0.02,0.02,0.00,0.0,0.0,0.00,0.00,0.00,...,0.00,0.01,0.01,0.0,0.00,0.04,0.0,0.0,0.00,0.01
4,1872382287_25,0.01,0.00,0.01,0.01,0.0,0.0,0.00,0.00,0.00,...,0.01,0.00,0.01,0.0,0.00,0.03,0.0,0.0,0.00,0.01
5,1872382287_30,0.00,0.02,0.02,0.00,0.0,0.0,0.00,0.00,0.01,...,0.00,0.00,0.01,0.0,0.00,0.04,0.0,0.0,0.01,0.02
6,1872382287_35,0.00,0.02,0.02,0.01,0.0,0.0,0.00,0.00,0.00,...,0.00,0.00,0.02,0.0,0.00,0.04,0.0,0.0,0.00,0.01
7,1872382287_40,0.00,0.02,0.01,0.00,0.0,0.0,0.00,0.00,0.00,...,0.00,0.00,0.02,0.0,0.00,0.05,0.0,0.0,0.00,0.01
8,1872382287_45,0.00,0.01,0.00,0.00,0.0,0.0,0.00,0.01,0.00,...,0.00,0.00,0.01,0.0,0.01,0.09,0.0,0.0,0.00,0.01
9,1872382287_50,0.00,0.02,0.01,0.01,0.0,0.0,0.00,0.00,0.00,...,0.00,0.00,0.01,0.0,0.00,0.05,0.0,0.0,0.00,0.01


In [63]:
sample_submission = pd.read_csv("/kaggle/input/birdclef-2024/sample_submission.csv")
assert set(sample_submission.columns) == set(submission_df.columns)
#submission_df = submission_df[sample_submission.columns]

In [ ]:
submission_df.to_csv('submission.csv', index=False)